In [21]:
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from splitter import *
from retriever import *

# Định nghĩa regex pattern cho các tiêu đề (sections)
patterns = [
    r'(?<=\n)Chương \d+: .+',  # Mục lớn như "Chương 1: TỔNG QUAN VỀ BAO BÌ"
    r'(?<=\n)\d+\.\d+ .+',  # Mục nhỏ hơn như "1.1 ĐẶC ĐIỂM CHUNG"
    r'(?<=\n)\d+\.\d+\.\d+ .+',  # Mục nhỏ hơn nữa như "1.1.1 Đặc điểm chung của bao bì"
    r'(?<=\n)\d+\.\d+\.\d+\.\d+ .+'  # Mục nhỏ hơn nữa như "1.1.2.1 Tính năng chứa đựng sản phẩm"
]

# Chia tài liệu thành các đoạn
loaders = TextLoader("doc/"+"spkt.txt", encoding = 'UTF-8')
docs = [loaders.load()]
splitter = PatternSplitter(patterns)
split_docs = splitter.split_documents(docs[0])
# In các đoạn
def beautiful_print(split_docs):
    for split_doc in split_docs:
        print(" ".join([split_doc.metadata['chapter'],split_doc.metadata['section'],split_doc.metadata['subsection'],split_doc.metadata['subsubsection'],]))
        if type(split_doc.metadata['summary']) is str:
            print(split_doc.metadata['summary'][:50])
        else:
            print(split_doc.metadata['summary'])
        print("----------")
        beautiful_print(split_doc.metadata["subtree"])

         
beautiful_print(split_docs)


<class 'list'>
before sum
Limit Token: Hệ thống sẽ chạy lại


In [20]:
from util import *
from retriever import *

question = "đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm B"
en_question = vn_2_en(question)
print(en_question)
retriever = TreeRetriever(documents=split_docs)
result = retriever.invoke(en_question)
print(result.page_content)

    characteristics of packaging from cardboard boxes type group B
Bao bì hộp từ giấy bìa: Là bao bì dạng hộp được sản xuất từ giấy bìa (giấy có định lượng lớn hơn 160 g/m²). Hộp bao gồm một hay nhiều mảnh được cấn và bế. Các đường bế tạo đường viền mảnh hộp, các đường cấn tạo đường nếp để gấp hộp, các mảnh bế sau đó có thể được gấp và dán. Các hộp sau khi gấp dán được lưu trữ và vận chuyển đến khách hàng ở thể phẳng, sau đó được dựng thành hộp trong quá trình đóng gói.
Các mảnh bế là các mảnh phẳng có các đường viền được cắt theo hình dạng trải phẳng của hộp. Các đường phân cách giữa các diện của hộp là các đường cần có nhiệm vụ tạo nếp gấp, các đường này là cơ sở cho việc dựng hộp từ khổ phẳng sang không gian ba chiều.
Do tính dễ vận chuyển, lưu trữ nên các dạng hộp hình hộp chữ nhật được sử dụng rộng rãi trong công nghiệp và cuộc sống. Ngoài bao bì dạng hình hộp chữ nhật, với đặc tính dễ định hình của giấy bìa và khả năng linh hoạt của quá trình cấn bế, và yêu cầu hấp dẫn người tiêu

In [10]:
print(result)

page_content='Xác định các thông tin cần thiết: - Thông tin về sản phẩm:
+ Kích thước và thể tích của sản phẩm bên trong
+ Sản phẩm có cần không gian để dịch chuyển, hay cứng, hoặc có thể nén được
+ Khối lượng sản phẩm
+ Cấp độ của bao bì
+ Số lượng sản phẩm trong bao bì
+ Sản phẩm là chất khí, rắn, chất lỏng, bột hay kem: Nếu là chất khí phải quan tâm đến thể tích, áp suất khối lượng riêng, nhiệt độ. Nếu là chất lỏng cần quan tâm đến độ nhớt, dính, khả năng tạo hơi, khối lượng riêng.
+ Tính chất dễ vỡ của sản phẩm và sự chống va đập cần thiết khi vận chuyển
+ Bị hỏng khi mài mòn, va đập
+ Có bị hư hỏng khi ẩm, có cần chống ẩm hay không
+ Có mùi hay không
+ Khả năng tạo khí
+ Các yêu cầu đặc biệt để bảo quản
+ Có dầu, mỡ...
+ Nhạy cảm với nhiệt độ quá cao hoặc quá thấp
+ Thay đổi màu sắc hay tính chất khi bị chiếu sáng
+ Tính chất hóa học của sản phẩm: tính axít, tính kiềm, hoạt tính hóa học...
- Thông tin về khách hàng
+ Ai là người sử dụng (phụ nữ, đàn ông, độ tuổi, nhóm người nào)
+

# SQL

In [57]:
from langchain_community.utilities import SQLDatabase

username = "postgres"
password = "12345678"
host = "localhost"
port = "5432"
database = "postgres"
postgres_uri = f"postgresql://{username}:{password}@{host}:{port}/{database}"
db = SQLDatabase.from_uri(postgres_uri)

In [76]:
import re
import os
import openai
import psycopg2
from PIL import Image
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda
from sqlalchemy.exc import ProgrammingError

openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)

def sql_gen(question):
    for i in range(5):
        write_query = create_sql_query_chain(llm, db)
        sql_code = write_query.invoke(question)
        if ": " in sql_code:
            sql_code = sql_code.strip().split(": ")[1]
        if "```sql" in sql_code:
            sql_code = sql_code.strip()[7:-4]
        if 'SQLQuery' in sql_code:
            return sql_code
        try:
            db.run(sql_code)
            break
        except ProgrammingError as e:
            print("Code gen: \n")
            print(sql_code)
            print("\nCode SQL gen bị sai, đang tiến hành gen lại")
        except Exception as e:
            print(e)
            break
    return sql_code

write_query  = RunnableLambda(sql_gen)
execute_query = QuerySQLDataBaseTool(db=db)

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}

Remember to note the photoes
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
)

answer_chain = (
    answer_prompt
    | llm
    | StrOutputParser()
)

question = "chiều ngang Bao giấy 25kg TAPX-8"

code = chain.invoke({"question": question})
print("code:", code)
pattern = r"img\\\\produces_property\\\\[^\s]+\.png"
matches = re.findall(pattern, str(code['result']))
if matches:
    matches.reverse()
    for idx, match in enumerate(matches):
        img = Image.open(match)
        img.show()
        cap = "Hình "+str(idx+1)
        code['result'] = code['result'].replace(match,cap)
try:
    db.run(code['query'])
    response =  .invoke(code)
    if "không phải là một câu lệnh SQL hợp lệ" in response:
        response = "Không có dữ liệu để truy vấn"
except Exception as e:
    print(e)
    response = "Không có dữ liệu để truy vấn"

print("Answer: "+str(response))

code: {'question': 'chiều ngang Bao giấy 25kg TAPX-8', 'query': 'SELECT "size_type", "length" \nFROM packaging_sizes \nWHERE "id" = \'BT1KH00051KP0001\' \nLIMIT 5;', 'result': "[('Chiều dài bao', '950'), ('Chiều ngang bao', '500'), ('Nẹp đáy bao', 'Chiều rộng: 60mm, Chiều dài: 620mm'), ('Túi PE', 'Chiều ngang: 520mm, Chiều dài: 1100 mm, Đáy: 5cm, Khối lượng: 30gr')]"}
Answer: Chiều ngang của bao giấy 25kg TAPX-8 là 500 mm.


In [28]:
create_sql_query_chain(llm, db).get_prompts()[0].pretty_print()

You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLR

In [82]:
from util import *
import ast
db = connect_database()
packagings_info = db.get_table_info(['packagings'])
packagings_samples = db.run('SELECT * FROM packagings;')
packagings_samples = ast.literal_eval(packagings_samples)
samples = ""
for line in packagings_samples[3:]:
    samples += "\t".join(line)+'\n'

sql_info = db.get_table_info([item for item in db.get_table_names() if item != 'packagings'])
print(packagings_info[:-2]+samples+"\n"+sql_info)


CREATE TABLE packagings (
	id VARCHAR(255) NOT NULL, 
	packaging_name VARCHAR(255), 
	industry VARCHAR(255), 
	raw_materials_contained VARCHAR(255), 
	mass VARCHAR(255), 
	storage_time VARCHAR(255), 
	market VARCHAR(255), 
	CONSTRAINT packagings_pkey PRIMARY KEY (id)
)

/*
3 rows from packagings table:
id	packaging_name	industry	raw_materials_contained	mass	storage_time	market
BT1KH00051KP0001	Bao giấy 25kg TAPX-8	Bột - Thực phẩm	Bột mì	25kg	12 tháng	Trong nước + Xuất khẩu
BT0KH00000KP0000	Bao giấy dán đáy 3 lớp giấy - dán nẹp đầu bao					
HN1KH00089KP0003	Bao hạt nhựa VEP 25KG (56x83)	Hạt nhựa	Hạt nhựa	25kg	12 tháng	Trong nước
XD2KH00073BP0002	Bao Keo dán gạch 25kg ( Hồng)	Phụ gia xây dựng	Keo dán gạch	25kg	12 tháng	Trong nước
BT1KH00050KP0001	Bao KP 25kg	Bột - Thực phẩm	Bột mì	25kg	12 tháng	Trong nước + Xuất khẩu
BT1KH00000KP0000	Bao giấy Dán Đáy 2 Đầu - 3 Lớp Giấy					
XD4KH00009BP0006	Bao Best Tile CE075 25kg	Phụ gia thực phẩm	Keo dán gạch	25kg	12 tháng	Trong nước
TA1KH00077OP0001

# Routing

In [65]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.utils.math import cosine_similarity

logic_query_prompt = """You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

Question: """
semantic_query_prompt = """Bạn là AI Chatbot của Công ty CP TM & SX Bao Bì Ánh Sáng hay BBAS. 
                    Bạn ở đây để trả lời tất cả các câu hỏi về văn hóa doanh nghiệp với các tài liệu bạn được cung cấp. 
                    Tài liệu được cung cấp: 
                    Nếu tài liệu được cung cấp là "Không có tài liệu liên quan đến câu hỏi hoặc yêu cầu này." hoặc các tài liệu được cung cấp không liên quan đến câu hỏi hoặc yêu cầu thì trả lời là \"Tôi không được cung cấp thông tin để trả lời câu hỏi này\"
                    Lưu ý: CÁC CHỈ TRẢ LỜI CÁC CÂU HỎI HOẶC YÊU CẦU ĐƯỢC HỎI, KHÔNG THỪA, KHÔNG THIẾU VÀ KHÔNG TỰ Ý TÓM TẮT HAY RÚT NGẮN CÂU TRẢ LỜI
                    Câu hỏi:"""
                    
prompt_templates = [logic_query_prompt, semantic_query_prompt]
prompt_embeddings = embeddings.embed_documents(prompt_templates)
query_embedding = embeddings.embed_query("Doanh nghiệp có bao nhiêu nhân viên")
similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
similarity

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [63]:
most_similar = prompt_templates[similarity.argmax()]
most_similar

'Bạn là AI Chatbot của Công ty CP TM & SX Bao Bì Ánh Sáng hay BBAS. \n                    Bạn ở đây để trả lời tất cả các câu hỏi về văn hóa doanh nghiệp với các tài liệu bạn được cung cấp. \n                    Tài liệu được cung cấp: \n                    Nếu tài liệu được cung cấp là "Không có tài liệu liên quan đến câu hỏi hoặc yêu cầu này." hoặc các tài liệu được cung cấp không liên quan đến câu hỏi hoặc yêu cầu thì trả lời là "Tôi không được cung cấp thông tin để trả lời câu hỏi này"\n                    Lưu ý: CÁC CHỈ TRẢ LỜI CÁC CÂU HỎI HOẶC YÊU CẦU ĐƯỢC HỎI, KHÔNG THỪA, KHÔNG THIẾU VÀ KHÔNG TỰ Ý TÓM TẮT HAY RÚT NGẮN CÂU TRẢ LỜI\n                    Câu hỏi:'

In [5]:
from util import *

db = connect_database()
question = "Chiều ngang bao  Bao giấy 25kg TAPX-8"
sql_info = db.get_table_info()
config = {"configurable": {"session_id": "phuccngo"}}
response = routing(question,sql_info, config)
print(response)
print(type(response))

Deleted message with ID: 66e170eee8d5477beadd878e
Deleted message with ID: 66e170eee8d5477beadd878f
True
<class 'str'>


In [1]:
#Evaluation routing
from util import *
questions = [
    ("nguyên liệu mà bao Vitec Grout chứa là gì","True"),
    ("chiều ngang bao  Bao giấy dán đáy 3 lớp giấy là bao nhiêu, cả ảnh minh họa","True"),
    ("nguyên liệu mà bao kpk không in chứa là gì","True"),
    ("Bao REP chứa được bao nhiêu ký","True"),
    ("thị trường tiêu thụ của loại bao NPK 20.5.15 Xanh Én này ở đâu","True"),
    ("nguyên liệu mà bao kpk không in chứa là gì","True"),
    ("quy trình sản xuất Bao giấy 25kg TAPX-8 gồm bao nhiêu bước","True"),
    ("thị trường tiêu thụ bao bột trét tường City ngoài trời","True"),
    ("ngành mà Bao hạt nhựa VEP 25KG được dùng","True"),
    ("thời gian bảo quản của bao Zucca Zeo","True"),
    ("ngành sử dụng bao Zucca Zeo là ngành nào","True"),
    ("nguyên liệu mà bao bì giấy Size L (Blue) chứa là gì","True"),
    ("bao đựng hạt nhựa TRIREX chứa bao nhiêu kí","True"),
    ("đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D","False"),
    ("trình bày sơ đồ quy trình sản xuất Bao giấy 25kg TAPX-8","True"),
    ("bao zucca zeo thì dùng chứa gì","True"),
    ("đặc điểm chung của bao bì","False"),
    ("tính năng chứa đựng của bao bì là gì","False"),
    ("bao KP đựng hạt nhựa ECOTHANE dùng trong ngành nào","True"),
    ("ngành công nghiệp sử dụng bao KP đựng hạt nhựa ECOTHANE  để chứa là ngành công nghiệp nào","True"),
    ("bao KP đựng hạt nhựa ECOTHANE 25kg  dùng để chứa gì","True"),
    ("dùng bao KP đựng hạt nhựa ECOTHANE 25kg  để chứa gì","True"),
    ("thị trường chủ yếu của bao KP đựng hạt nhựa ECOTHANE","True"),
    ("dung sai chiều dài bao giấy dán đáy 3 lớp giấy là bao nhiêu","True"),
    ("tính năng bảo vệ sản phẩm của bao bì thì sao","False"),
    ("bao bì vận chuyển là gì","False"),
    ("các loại bao bì khi phân loại theo vật liệu","False"),
    ("nói chi tiết hơn về bao bì từ giấy","False"),
     ]
config = {"configurable": {"session_id": "phuccngo"}}
db = connect_database()
sql_info = get_table_info_with_full_packagings_samples(db)
correct = 0
for idx, question in enumerate(questions):
    response = routing(question[0],sql_info, config)
    if response == question[1]:
        correct += 1
    print(idx,": ",response,", ",correct)
correct*100/len(questions)

c:\D\my project\Product_chatbot\util.py:224: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 0.3.0. Use get_usable_table_names instead.
  sql_info = db.get_table_info([item for item in db.get_table_names() if item != 'packagings'])


0 :  True ,  1
1 :  True ,  2
2 :  True ,  3
3 :  True ,  4
4 :  False ,  4
5 :  True ,  5
6 :  True ,  6
7 :  False ,  6
8 :  True ,  7
9 :  False ,  7
10 :  False ,  7
11 :  True ,  8
12 :  True ,  9
13 :  False ,  10
14 :  True ,  11
15 :  False ,  11
16 :  False ,  12
17 :  False ,  13
18 :  False ,  13
19 :  False ,  13
20 :  False ,  13
21 :  False ,  13
22 :  False ,  13
23 :  False ,  13


54.166666666666664

In [2]:
sql_info

'\nCREATE TABLE packagings (\n\tid VARCHAR(255) NOT NULL, \n\tpackaging_name VARCHAR(255), \n\tindustry VARCHAR(255), \n\traw_materials_contained VARCHAR(255), \n\tmass VARCHAR(255), \n\tstorage_time VARCHAR(255), \n\tmarket VARCHAR(255), \n\tCONSTRAINT packagings_pkey PRIMARY KEY (id)\n)\n\n/*\n3 rows from packagings table:\nid\tpackaging_name\tindustry\traw_materials_contained\tmass\tstorage_time\tmarket\nBT1KH00051KP0001\tBao giấy 25kg TAPX-8\tBột - Thực phẩm\tBột mì\t25kg\t12 tháng\tTrong nước + Xuất khẩu\nBT0KH00000KP0000\tBao giấy dán đáy 3 lớp giấy - dán nẹp đầu bao\t\t\t\t\t\nHN1KH00089KP0003\tBao hạt nhựa VEP 25KG (56x83)\tHạt nhựa\tHạt nhựa\t25kg\t12 tháng\tTrong nước\nXD2KH00073BP0002\tBao Keo dán gạch 25kg ( Hồng)\tPhụ gia xây dựng\tKeo dán gạch\t25kg\t12 tháng\tTrong nước\nBT1KH00050KP0001\tBao KP 25kg\tBột - Thực phẩm\tBột mì\t25kg\t12 tháng\tTrong nước + Xuất khẩu\nBT1KH00000KP0000\tBao giấy Dán Đáy 2 Đầu - 3 Lớp Giấy\t\t\t\t\t\nXD4KH00009BP0006\tBao Best Tile CE075 25kg

# delete db

In [8]:
#Kiểm tra toàn bộ cơ sở dữ liệu
from pymongo import MongoClient
query = {'SessionId': 'phuccngo'}
dbclient = MongoClient('mongodb://localhost:27017/')
db = dbclient['product_chat_database']
conversations_collection = db['messages']
result = conversations_collection.count_documents(query)
result

23

In [1]:
from pymongo import MongoClient
from datetime import datetime

# MongoDB connection details
mongo_uri = 'mongodb://localhost:27017/'
database_name = 'product_chat_database'
collection_name = 'messages'

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[database_name]
collection = db[collection_name]

def delete_most_recent_message():
    mongo_uri = 'mongodb://localhost:27017/'
    database_name = 'product_chat_database'
    collection_name = 'messages'
    client = MongoClient(mongo_uri)
    db = client[database_name]
    collection = db[collection_name]
    try:
        # Retrieve the most recent message
        most_recent_message = collection.find_one(sort=[('timestamp', -1)])
        
        if most_recent_message:
            most_recent_message_id = most_recent_message['_id']

            # Delete the most recent message
            collection.delete_one({'_id': most_recent_message_id})
            print(f"Deleted message with ID: {most_recent_message_id}")
        else:
            print("No messages found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the MongoDB connection
        client.close()

# Usage
delete_most_recent_message()

No messages found.


# Query Transformations

Hiện tại chưa được sử dụng vì chưa được cung cấp dữ liệu của db để chuẩn hóa câu hỏi. Và nếu được cung cấp thì có nguy cơ sẽ chuẩn hóa theo db sql không theo hướng tài liệu khác, gây ra sự không chính xác

In [78]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
examples = [
    {
        "input": "hãy kể cho tôi chi tiết các bước của quy trình sản xuất của Bao dán đáy 3 lớp giấy",
        "output": "Chi tiết các bước quy trình sản xuất bao dán đáy 3 lớp giấy",
    },
    {
        "input": "số bước trong quy trình sản xuất Bao giấy 25kg TAPX-8",
        "output": "có bao nhiêu bước trong quy trình sản xuất bao giấy 25kg TAPX-8",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Bạn là một chuyên gia về kiến thức toàn cầu. Nhiệm vụ của bạn là tạo ra một câu hỏi dựa trên lịch sử tin nhắn và câu hỏi hiện tại để dễ dàng trả lời hơn.
            LƯU Ý: HÃY GIỮ LẠI CÁC CHỦ NGỮ ĐÃ ĐƯỢC SỬ DỤNG Ở CÂU HỎI
            Lịch sử tin nhắn:
            {history}
            Đây là 1 số ví dụ:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [79]:
from langchain_core.messages import (
    trim_messages,
)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter
from util import *

llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
token_counter_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)
trimmer = trim_messages(
    max_tokens=500,
    strategy="last",
    token_counter=token_counter_model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
chain = (
    RunnablePassthrough.assign(messages=itemgetter("question") | trimmer)
    | (lambda output: (output.update({'question': output['messages']}), output)[-1])
    | (lambda output: (
        output.update({'history': output['question'][:-1]}),
        output.update({'question': output['question'][-1]}),
        print(output['history']),
        output)[-1])
    | prompt
    | llm
    | StrOutputParser()
)
generate_queries_step_back = RunnableWithMessageHistory(
    chain,
    get_session_history_mongodb,
    input_messages_key="question", #Which key is user's input
)


In [81]:
question = "đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D"
config = {"configurable": {"session_id": "phuccngo"}}
result = generate_queries_step_back.invoke(
    {
        "question": question
    },
    config=config,
)
delete_two_most_recent_message(config['configurable']['session_id'])
result

[HumanMessage(content='quy trình sản xuất của  Bao dán đáy 3 lớp giấy gồm bao nhiêu bước'), AIMessage(content='Quy trình sản xuất của bao dán đáy 3 lớp giấy gồm 2 bước.')]
-----messages-----
quy trình sản xuất của  Bao dán đáy 3 lớp giấy gồm bao nhiêu bước
Quy trình sản xuất của bao dán đáy 3 lớp giấy gồm 2 bước.
đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D
đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D là gì?
--------out--------
Deleted message with message: đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D là gì?
Deleted message with message: đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D


'đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm D là gì?'